# AutoImputation

This notebook demonstrates the functionality of the `autoimpute` module, which provides an automated approach to selecting and applying optimal imputation methods for missing data. Rather than manually testing different approaches, AutoImpute evaluates multiple methods, identifies which performs best for your specific data, and applies it to generate high-quality imputations.

In [36]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.datasets import load_diabetes
import warnings

warnings.filterwarnings("ignore")

# Set pandas display options to limit table width
pd.set_option("display.width", 600)
pd.set_option("display.max_columns", 10)
pd.set_option("display.expand_frame_repr", False)

from microimpute.comparisons.autoimpute import autoimpute
from microimpute.comparisons.plot import plot_autoimpute_method_comparison

## Data preparation

This demonstration uses the diabetes dataset from scikit-learn. In real-world imputation scenarios, you would typically have a "donor" dataset with complete information for both predictor and target variables, and a "receiver" dataset that lacks some target variables that need to be imputed.

In [18]:
# Load the diabetes dataset
diabetes = load_diabetes()
diabetes_data = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)

# Display the first few rows to understand the data structure
diabetes_data.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641


For this demonstration, the diabetes dataset is split into donor and receiver portions. Part of the data is treated as the donor dataset with complete information, and another part as the receiver dataset with some variables that need imputation.

In [19]:
# Split the data into donor and receiver portions
donor_indices = np.random.choice(len(diabetes_data), size=int(0.7*len(diabetes_data)), replace=False)
receiver_indices = np.array([i for i in range(len(diabetes_data)) if i not in donor_indices])

donor_data = diabetes_data.iloc[donor_indices].reset_index(drop=True)
receiver_data = diabetes_data.iloc[receiver_indices].reset_index(drop=True)

# Define which variables we'll use as predictors and which we want to impute
predictors = ['age', 'sex', 'bmi', 'bp']
imputed_variables = ['s1', 's4']

# For demonstration purposes, we'll remove the variables we want to impute from the receiver dataset
receiver_data_without_targets = receiver_data.drop(columns=imputed_variables)

print(f"Donor data shape: {donor_data.shape}")
print(f"Receiver data shape: {receiver_data_without_targets.shape}")
print(f"Predictors: {predictors}")
print(f"Variables to impute: {imputed_variables}")

Donor data shape: (309, 10)
Receiver data shape: (133, 8)
Predictors: ['age', 'sex', 'bmi', 'bp']
Variables to impute: ['s1', 's4']


## Running AutoImpute

Use the `autoimpute` function to automatically evaluate different imputation methods, select the best one, and generate imputations. The function handles all the complexity of model evaluation, selection, and application in a single call.

In [32]:
# Run the autoimpute process
imputations, imputed_data, fitted_model, method_results_df = autoimpute(
    donor_data=donor_data,
    receiver_data=receiver_data_without_targets,
    predictors=predictors,
    imputed_variables=imputed_variables,
    k_folds=5  # Number of cross-validation folds
)

print(f"Shape of receiver data before imputation: {receiver_data_without_targets.shape} \nShape of receiver data after imputation: {imputed_data.shape}")

Shape of receiver data before imputation: (133, 8) 
Shape of receiver data after imputation: (133, 10)


## Understanding the results

The `autoimpute` function returns four key objects that provide comprehensive information about the imputation process:

- `imputations`: A dictionary where keys are quantiles used for imputation and values are DataFrames containing the imputed values at each quantile
- `imputed_data`: The receiver dataset with imputed values integrated into it
- `fitted_model`: The best-performing imputation model, already fitted on the donor data
- `method_results_df`: A DataFrame with detailed performance metrics for all evaluated imputation methods

In [21]:
# Examine the comparative performance of different imputation methods
print("Cross-validation results for different imputation methods:")
method_results_df

Cross-validation results for different imputation methods:


,0.05,0.1,0.15,0.2,0.25,...,0.8,0.85,0.9,0.95,mean_loss
QRF,0.090426,0.173055,0.267954,0.346141,0.410739,...,0.324274,0.301059,0.234717,0.133190,0.375195
OLS,0.082278,0.139667,0.189369,0.233855,0.271179,...,0.282606,0.241975,0.188732,0.116327,0.272463
QuantReg,0.081522,0.136159,0.187598,0.236638,0.269456,...,0.282611,0.244639,0.191277,0.121408,0.273492
Matching,0.535175,0.537145,0.539115,0.541085,0.543055,...,0.564723,0.566693,0.568663,0.570633,0.552904


The table above provides a comprehensive view of how each imputation method performs across different quantiles. The 'mean_loss' column shows the average quantile loss across all quantiles for each method. Lower values indicate better performance, and AutoImpute automatically selects the method with the lowest average loss.

In [22]:
# Identify which method was selected as the best performer
best_method = method_results_df['mean_loss'].idxmin()
print(f"Best performing method: {best_method}")
print(f"Average loss: {method_results_df.loc[best_method, 'mean_loss']:.4f}")

Best performing method: OLS
Average loss: 0.2725


## Visualizing method comparison

Visualize how different methods perform across quantiles provides insight into which methods are most appropriate for different parts of the distribution.

In [23]:
# Extract the quantiles used in the evaluation
quantiles = [q for q in method_results_df.columns if isinstance(q, float)]

# Generate a visualization comparing methods across quantiles
fig = plot_autoimpute_method_comparison(
    method_results_df=method_results_df,
    quantiles=quantiles,
    metric_name="Quantile Loss"
)
fig.show()

The plot above illustrates how each imputation method performs across different quantiles of the distribution. Methods with consistently lower lines generally perform better overall.

## Examining the imputed values

Now let us assess the actual imputed values generated by the best-performing method.

In [24]:
# Examine imputed values at the median (0.5 quantile)
median_imputations = imputations[0.5]
print("Median imputed values:")
median_imputations.head()

Median imputed values:


,s1,s4
0,-0.007898,-0.027792
1,-0.021793,-0.028920
2,-0.006741,0.005205
3,0.007089,0.031084
4,-0.013832,-0.014011


In [25]:
# Look at the full receiver dataset with imputed values integrated
print("Receiver dataset with imputed values:")
imputed_data.head()

Receiver dataset with imputed values:


,age,sex,bmi,bp,s2,s3,s5,s6,s1,s4
0,-0.001882,-0.044642,-0.051474,-0.026328,-0.019163,0.074412,-0.068332,-0.092204,-0.007898,-0.027792
1,-0.092695,-0.044642,-0.040696,-0.019442,-0.079288,0.041277,-0.041176,-0.096346,-0.021793,-0.028920
2,-0.070900,-0.044642,0.039062,-0.033213,-0.034508,-0.024993,0.067737,-0.013504,-0.006741,0.005205
3,0.027178,0.050680,0.017506,-0.033213,0.045972,-0.065491,-0.096435,-0.059067,0.007089,0.031084
4,-0.085430,-0.044642,-0.004050,-0.009113,0.007767,0.022869,-0.061176,-0.013504,-0.013832,-0.014011


## Evaluating imputation quality

In this demonstration, since the receiver dataset was artificially created by removing variables from the original data, there exists the unique opportunity to evaluate the quality of our imputations by comparing them to the actual values.

In [26]:
# Visualize comparison between actual and imputed values
for var in imputed_variables:
    fig = go.Figure()
    
    # Plot actual values
    fig.add_trace(go.Scatter(
        x=receiver_data.index,
        y=receiver_data[var],
        mode='markers',
        name='Actual Values',
        marker=dict(color='blue', size=8)
    ))
    
    # Plot imputed values
    fig.add_trace(go.Scatter(
        x=imputed_data.index,
        y=imputed_data[var],
        mode='markers',
        name='Imputed Values',
        marker=dict(color='red', size=8)
    ))
    
    # Customize the plot appearance
    fig.update_layout(
        title=f'Comparison of Actual vs Imputed Values for {var}',
        xaxis_title='Sample Index',
        yaxis_title=f'{var} Value',
        legend_title='Type',
        hovermode='closest'
    )
    
    fig.show()

The plots above show how well the imputed values (red) match the actual values (blue) that were removed from the receiver dataset. This visual comparison helps assess the quality of the imputations generated by the best-performing method.

## Advanced usage: custom models and hyperparameters

The AutoImpute function allows for customization of both the models to evaluate and their hyperparameters. This flexibility enables adaptation to specific dataset characteristics and imputation requirements.

In [34]:
from microimpute.models import QRF, OLS, QuantReg

# Specify a custom subset of models to evaluate
custom_models = [QRF, OLS, QuantReg]

# Specify custom hyperparameters for some models
custom_hyperparameters = {"QRF": {"n_estimators": 200, "max_depth": 10}}

# Then simply run autoimpute with custom models and hyperparameters
advanced_imputations, advanced_imputed_data, advanced_fitted_model, advanced_method_results = autoimpute(
    donor_data=donor_data,
    receiver_data=receiver_data_without_targets,
    predictors=predictors,
    imputed_variables=imputed_variables,
    models=custom_models,
    hyperparameters=custom_hyperparameters,
    k_folds=5
)

advanced_imputations[0.5]

,s1,s4
0,-0.007898,-0.027792
1,-0.021793,-0.028920
2,-0.006741,0.005205
3,0.007089,0.031084
4,-0.013832,-0.014011
...,...,...
128,-0.011857,-0.022794
129,0.012390,0.011905
130,0.012585,0.015373
131,-0.014099,-0.032634
